In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext("local",'app')
spark = SparkSession.builder.appName('name').config('spark.sql.shuffle.partitions',10).getOrCreate()

In [3]:
df = spark.read.csv('../data/sales.csv', header=True, inferSchema=True)

In [4]:
df.printSchema()

root
 |-- Transaction_date: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Account_Created: string (nullable = true)
 |-- Last_Login: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)



## Q1 Find all distinct countries.

Hint: use select(), distinct()

In [5]:
df_distinct_country=df.select('Country').distinct()
df.select('Country').distinct().show()

+--------------------+
|             Country|
+--------------------+
|              France|
|             Denmark|
|             Austria|
|         New Zealand|
|              Canada|
|             Germany|
|              Kuwait|
|               Malta|
|         South Korea|
|             Finland|
|              Jersey|
|              Israel|
|        South Africa|
|              Norway|
|United Arab Emirates|
|             Iceland|
|      United Kingdom|
|             Ireland|
|             Belgium|
|             Moldova|
+--------------------+
only showing top 20 rows



## Question 2
Find the Name and Price of sales records in Brazil.

Hint: use filter().

In [8]:
df.select("Name","Price").filter("Country='Brazil'").show()

+-------+-----+
|   Name|Price|
+-------+-----+
|Joachim| 1200|
|  Diana| 7500|
+-------+-----+



##  Question 3

For each country, find the total Price.

Hint: Use groupBy()

In [9]:
df.groupBy('Country').sum("Price").show()

+--------------------+----------+
|             Country|sum(Price)|
+--------------------+----------+
|              France|     30300|
|             Denmark|      8400|
|             Austria|      3600|
|         New Zealand|      2400|
|              Canada|     42000|
|             Germany|     22800|
|              Kuwait|      1200|
|               Malta|      3600|
|         South Korea|      1200|
|             Finland|      1200|
|              Jersey|      1200|
|              Israel|      1200|
|        South Africa|      3600|
|              Norway|     12000|
|United Arab Emirates|      3600|
|             Iceland|      1200|
|      United Kingdom|     63600|
|             Ireland|     29100|
|             Belgium|      3600|
|             Moldova|      1200|
+--------------------+----------+
only showing top 20 rows



## Question 4

List countries by their total Price in descending order.

Hint: Use orderBy()



In [14]:
df.groupBy('Country').sum("Price").orderBy('sum(Price)',ascending=False).show()

+--------------------+----------+
|             Country|sum(Price)|
+--------------------+----------+
|       United States|    350350|
|      United Kingdom|     63600|
|              Canada|     42000|
|              France|     30300|
|             Ireland|     29100|
|             Germany|     22800|
|           Australia|     22800|
|         Switzerland|     19200|
|         Netherlands|     14400|
|              Norway|     12000|
|              Brazil|      8700|
|              Sweden|      8400|
|             Denmark|      8400|
|               Malta|      3600|
|             Austria|      3600|
|             Belgium|      3600|
|United Arab Emirates|      3600|
|        South Africa|      3600|
|               Italy|      2400|
|              Turkey|      2400|
+--------------------+----------+
only showing top 20 rows



## Question 5
Redo Question 3, but replace the country names by their IDs.

Hint: Use join()

In [16]:
df2 = spark.read.csv('../data/countries.csv', header=True, inferSchema=True)

In [17]:
df.join(df2,'Country')\
    .select('ID','Price')\
    .groupBy('ID')\
    .sum('Price')\
    .withColumnRenamed('sum(Price)', 'TotalPrice')\
    .show()

+---+----------+
| ID|TotalPrice|
+---+----------+
| 22|      3600|
| 32|      1200|
|  3|     22800|
| 29|      1200|
| 30|      1200|
| 34|      2400|
|  4|      1200|
| 10|      3600|
| 18|      2400|
| 25|      1200|
| 31|      1200|
|  1|     63600|
|  8|     42000|
| 11|      1200|
| 20|      1200|
| 21|      2400|
| 33|      8700|
|  2|    350350|
| 12|     19200|
| 28|      3600|
+---+----------+
only showing top 20 rows



## Question 6
Rewrite the PageRank example using DataFrame API.  Here is a skeleton of the code.  Your job is to fill in the missing part.  

In [4]:
# question code
from pyspark.sql.functions import *

numOfIterations = 10
lines = spark.read.text("../data/pagerank_data.txt")

a = lines.select(split(lines[0],' '))
links = a.select(a[0][0].alias('src'), a[0][1].alias('dst'))
outdegrees = links.groupBy('src').count()
ranks = outdegrees.select('src', lit(1).alias('rank'))


In [26]:
outdegrees.show()

+---+-----+
|src|count|
+---+-----+
|  1|    2|
|  4|    1|
|  3|    1|
|  2|    2|
+---+-----+



In [ ]:
for iteration in range(numOfIterations):
# FILL IN THIS PART
    
ranks.orderBy(desc('rank')).show()

In [5]:
for iteration in range(numOfIterations):
# FILL IN THIS PART
    distribution=links.join(outdegrees,"src").join(ranks,"src")\
        .select((links['dst']).alias("src"),\
        (1.0*ranks['rank']/outdegrees['count']).alias("distributions"))    
    result=distribution.groupBy("src").sum('distributions')
    ranks=result.select('src',(result['sum(distributions)']*0.85+0.15).alias('rank'))
ranks.orderBy(desc('rank')).show()

+---+------------------+
|src|              rank|
+---+------------------+
|  1|1.2981882732854677|
|  4|0.9999999999999998|
|  3|0.9999999999999998|
|  2|0.7018117267145316|
+---+------------------+

